In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [0]:
#We are going to scrape ALL players' stats from the website
url = "https://www.premierleague.com/clubs"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

#Get ALL team's link
teamLinks=[]
for elem in soup.find_all('a', href=re.compile('^/clubs/')):
    #print("https://www.premierleague.com" + elem['href'])
    temp = "https://www.premierleague.com" + elem['href']
    #replace 'overview' with 'squad'
    temp = temp.replace("overview", "squad")
    teamLinks.append(temp)
#show sample content of teamLinks
teamLinks[:2]

['https://www.premierleague.com/clubs/1/Arsenal/squad',
 'https://www.premierleague.com/clubs/2/Aston-Villa/squad']

In [0]:
#Get the player links from each team links
soup = []
playerLinks=[]
for i in range(len(teamLinks)):
    html = urlopen(teamLinks[i])
    soup = BeautifulSoup(html, 'lxml')
    for elem in soup.find_all('a', href=re.compile('^/players/')):
        #print("https://www.premierleague.com" + elem['href'])
        temp = "https://www.premierleague.com" + elem['href']
        temp = temp.replace("overview", "stats")
        playerLinks.append(temp)
#show content of playerLinks
print(len(playerLinks))
playerLinks[:2]

586


['https://www.premierleague.com/players/4985/Bernd-Leno/stats',
 'https://www.premierleague.com/players/4245/Emiliano-Martínez/stats']

In [0]:
#Get the ALL the attributes labels
url = 'https://www.premierleague.com/players/15202/Declan-Rice/stats'
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')
pstats = soup.find_all('span')
colattrib=[]
colattrib.append('player_name')
for elem in pstats:
    temp1 = elem.get_attribute_list('data-stat')[0]
    if type(temp1)!=type(None):
        colattrib.append(temp1)
#show content of the attributes label name
print(len(colattrib))
colattrib[:10]

40


['player_name',
 'appearances',
 'goals',
 'wins',
 'losses',
 'clean_sheet',
 'goals_conceded',
 'total_tackle',
 'won_tackle',
 'last_man_tackle']

In [0]:
#Get player's name and stats
#This process will take some time arround 5 minutes
#You can modify the range if you just want to get a sample
soup = []
colstats = []
col_all= []
col_all = []
temp = []
for i in range(len(playerLinks)):
#for i in range(5):
    temp = []
    #Use this if the link cotains non ASCII characters
    html = requests.get(playerLinks[i]).text
    soup = BeautifulSoup(html, 'lxml')
    pstats = soup.find_all('span')
    #name
    temp.insert(0, soup.select_one("div[class*='name t-colour']").text)
    #appearance
    #temp.insert(1, soup.select_one("span[class*='appearances']").get_text(' ', re.compile('\d+')))
    #goals
    #temp.insert(1, soup.select_one("span[class*='goals']").get_text(' ', re.compile('\d+')))
    for elem in range(1,len(colattrib)):
    #for att in range(5):
        test = "span[class*='" + colattrib[elem] + "']"
        #print(test)
        if soup.select_one(str(test))!=None:
            temp.insert(elem, soup.select_one(str(test)).get_text(' ', re.compile('\d+')))
            #print(soup.select_one(str(test)).get_text(' ', re.compile('\d+')))
        else:
            temp.insert(elem, '0')
    colstats.append(temp)


In [0]:
#Convert to Pandas DataFrame
df= pd.DataFrame(colstats)
df

In [0]:
#Adding column name
df.columns = colattrib

In [0]:
#Set index
df.set_index('player_name',inplace=True)
df

In [0]:
#Reset index
df.reset_index(inplace=True)
df

In [0]:
#Try to clean duplicate columns
df_clean = df.loc[:,~df.columns.duplicated()]

In [0]:
df_clean

,player_name,appearances,goals,wins,losses,clean_sheet,goals_conceded,total_tackle,won_tackle,last_man_tackle,blocked_scoring_att,interception,total_clearance,effective_head_clearance,clearance_off_line,ball_recovery,duel_won,duel_lost,won_contest,aerial_won,aerial_lost,own_goals,error_lead_to_goal,goal_assist,total_pass,big_chance_created,total_cross,accurate_cross,total_through_ball,accurate_long_balls,yellow_card,red_card,fouls,total_offside,att_hd_goal,att_rf_goal,att_lf_goal,hit_woodwork
0,Bernd Leno,43,57,21,10,8,57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,"1,192",0,0,0,0,183,2,0,0,0,0,0,0,0
1,Emiliano Martínez,6,7,3,3,3,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,124,0,0,0,0,28,0,0,1,0,0,0,0,0
2,Matt Macey,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Héctor Bellerín,143,6,82,32,49,147,193,0,1,30,187,279,133,3,670,552,625,177,145,197,1,1,18,"6,457",27,353,0,30,130,20,0,107,7,0,4,2,3
4,Sokratis,36,2,18,9,7,48,56,0,1,2,36,150,68,0,146,184,134,6,91,65,0,1,2,"1,930",4,0,0,1,137,14,0,45,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,Patrick Cutrone,8,1,2,2,0,0,3,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,48,1,5,0,0,0,0,0,5,3,0,0,1,0
582,Pedro Neto,7,0,1,1,0,0,1,0,0,2,1,1,0,0,0,0,0,0,0,0,0,0,1,62,2,6,0,0,0,1,0,1,0,0,0,0,0
583,Benny Ashley-Seal,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
584,Austin Samuels,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#Download your hardworking scrapped data when finished
from google.colab import files

df_clean.to_csv('df_clean.csv')
files.download('df_clean.csv')